# RBAC Endpoints Testing

This notebook allows you to test all RBAC (Role-Based Access Control) endpoints interactively.

## Endpoints Covered:
- `/api/rbac/roles/` - List/Create roles
- `/api/rbac/roles/{id}/` - Retrieve/Update/Delete role
- `/api/rbac/permissions/` - List/Create permissions
- `/api/rbac/permissions/{id}/` - Retrieve/Update/Delete permission
- `/api/rbac/user-roles/` - List/Create user role assignments
- `/api/rbac/user-roles/{id}/` - Retrieve/Update/Delete user role
- `/api/rbac/current-user/` - Get current user's RBAC info

## Setup

In [ ]:
import requests
import json
from pprint import pprint

BASE_URL = "http://localhost:8000"

# You need to login first to get tokens
access_token = None

def print_response(response):
    """Pretty print response"""
    print(f"Status Code: {response.status_code}")
    print(f"\nResponse Body:")
    try:
        pprint(response.json())
    except:
        print(response.text)

def get_headers():
    """Get authorization headers"""
    if not access_token:
        print("⚠️  Warning: No access token set!")
        return {}
    return {"Authorization": f"Bearer {access_token}"}

print("✅ Setup complete!")

## Login to Get Access Token

In [ ]:
# Login as admin
login_data = {
    "email": "admin@replycompass.com",
    "password": "admin123"
}

response = requests.post(f"{BASE_URL}/api/auth/login/", json=login_data)

if response.status_code == 200:
    data = response.json()
    access_token = data.get('access')
    print("✅ Login successful!")
    print(f"Access token: {access_token[:50]}...")
else:
    print("❌ Login failed!")
    print_response(response)

## 1. List All Roles

Get all roles in the system

In [ ]:
response = requests.get(
    f"{BASE_URL}/api/rbac/roles/",
    headers=get_headers()
)

print_response(response)

## 2. Get Specific Role Details

Get details of a specific role including permissions

In [ ]:
# Get role ID 1 (usually Guest)
role_id = 1

response = requests.get(
    f"{BASE_URL}/api/rbac/roles/{role_id}/",
    headers=get_headers()
)

print_response(response)

## 3. Create New Role

Create a custom role

In [ ]:
new_role_data = {
    "name": "Custom Role",
    "description": "A custom role for testing",
    "level": 50,
    "is_active": True
}

response = requests.post(
    f"{BASE_URL}/api/rbac/roles/",
    json=new_role_data,
    headers=get_headers()
)

print_response(response)

if response.status_code == 201:
    created_role_id = response.json().get('id')
    print(f"\n✅ Role created with ID: {created_role_id}")

## 4. Update Role

Update an existing role

In [ ]:
# Update the role we just created
role_id = 10  # Change this to the created role ID

update_data = {
    "name": "Custom Role Updated",
    "description": "Updated description",
    "level": 55
}

response = requests.patch(
    f"{BASE_URL}/api/rbac/roles/{role_id}/",
    json=update_data,
    headers=get_headers()
)

print_response(response)

## 5. List All Permissions

Get all available permissions

In [ ]:
response = requests.get(
    f"{BASE_URL}/api/rbac/permissions/",
    headers=get_headers()
)

print_response(response)

## 6. Get Specific Permission Details

In [ ]:
permission_id = 1

response = requests.get(
    f"{BASE_URL}/api/rbac/permissions/{permission_id}/",
    headers=get_headers()
)

print_response(response)

## 7. Create New Permission

In [ ]:
new_permission_data = {
    "name": "custom.test",
    "description": "A custom permission for testing",
    "resource": "custom",
    "action": "test"
}

response = requests.post(
    f"{BASE_URL}/api/rbac/permissions/",
    json=new_permission_data,
    headers=get_headers()
)

print_response(response)

## 8. List User Role Assignments

Get all user-role assignments

In [ ]:
response = requests.get(
    f"{BASE_URL}/api/rbac/user-roles/",
    headers=get_headers()
)

print_response(response)

## 9. Assign Role to User

Create a new user-role assignment

In [ ]:
# Assign role to user
assignment_data = {
    "user": 1,  # User ID
    "role": 2   # Role ID (e.g., User role)
}

response = requests.post(
    f"{BASE_URL}/api/rbac/user-roles/",
    json=assignment_data,
    headers=get_headers()
)

print_response(response)

## 10. Get Current User's RBAC Info

Get roles and permissions for the currently authenticated user

In [ ]:
response = requests.get(
    f"{BASE_URL}/api/rbac/current-user/",
    headers=get_headers()
)

print_response(response)

## 11. Filter Roles by Level

Get roles with specific level filtering

In [ ]:
# Get roles with level >= 50
response = requests.get(
    f"{BASE_URL}/api/rbac/roles/?level__gte=50",
    headers=get_headers()
)

print_response(response)

## 12. Search Permissions

Search permissions by name or resource

In [ ]:
# Search for user-related permissions
response = requests.get(
    f"{BASE_URL}/api/rbac/permissions/?search=user",
    headers=get_headers()
)

print_response(response)

## 13. Delete Role

⚠️ **Warning:** This will permanently delete the role

In [ ]:
# Delete the custom role (be careful!)
role_id = 10  # Change this to the role ID you want to delete

response = requests.delete(
    f"{BASE_URL}/api/rbac/roles/{role_id}/",
    headers=get_headers()
)

print_response(response)

if response.status_code == 204:
    print("\n✅ Role deleted successfully!")

## 14. Comprehensive Role Analysis

Get a complete overview of all roles and their permissions

In [ ]:
response = requests.get(
    f"{BASE_URL}/api/rbac/roles/",
    headers=get_headers()
)

if response.status_code == 200:
    roles = response.json()
    print(f"Total Roles: {len(roles)}\n")
    print("=" * 80)
    
    for role in roles:
        print(f"\n📋 Role: {role['name']} (Level: {role['level']})")
        print(f"   Description: {role['description']}")
        print(f"   Active: {role['is_active']}")
        print(f"   Permissions: {len(role.get('permissions', []))}")
        
        if role.get('permissions'):
            print("   Permission List:")
            for perm in role['permissions'][:5]:  # Show first 5
                print(f"     - {perm}")
            if len(role['permissions']) > 5:
                print(f"     ... and {len(role['permissions']) - 5} more")
        print("-" * 80)
else:
    print_response(response)